In [28]:
import requests # Importamos requests para hacer peticiones a la web
import json # Importamos json para manejar datos en formato JSON
from IPython.display import JSON # Esto sirve para mostrar JSON bonito en Jupyter
from keys import * # Traemos la clave de acceso desde el archivo keys.py

In [29]:
#be careful not to run too often as there is a maximum of 200 api calls per month

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/LIRF/2025-09-01T08:00/2025-09-01T20:00" # URL que consulta vuelos que llegan al aeropuerto EDDB entre las 8:00 y 20:00 del 1 de septiembre de 2025

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"} # Parámetros extra para la consulta: queremos ver vuelos cancelados, compartidos, de carga, privados, etc.

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    } # Encabezados necesarios para conectarse a la API de RapidAPI

response = requests.request("GET", url, headers=headers, params=querystring) # Hacemos la petición GET con todos los datos anteriores

In [30]:
if response.status_code == 200: 
# Si la respuesta fue exitosa (código 200), guardamos los datos
    new_data = response.json()

    try: # Intentamos abrir el archivo data.json para agregarle los nuevos datos
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []
# Agregamos los nuevos datos a la lista
    existing_data.append(new_data)
 # Guardamos todo de nuevo en el archivo data.json
    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:# Si algo salió mal, mostramos el código de error
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [31]:
flight_arrivals = response.json() # Guardamos la respuesta en una variable para trabajar con ella
# Sacamos algunos datos clave del primer vuelo (esto no guarda nada, solo accede)
for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [32]:
import pandas as pd # Importamos pandas para trabajar con tablas y numpy para manejar valores nulos
from numpy import nan

flight_arrivals = response.json()

def flight_extraction(flights):
    flight_data = {
        'scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from': flights['departure']['airport']['name'],
        'airline': flights['airline']['name'],
        'aircraft': flights.get('aircraft', {}).get('model', 'No disponible')
    }
    return pd.json_normalize(flight_data)


# Aplicamos la función a todos los vuelos y los juntamos en un solo DataFrame
flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)
# Mostramos las primeras filas para ver cómo quedó la tabla
flight_arrivals.head()

,flight_number,from,airline,aircraft,scheduled_arrival_time.utc,scheduled_arrival_time.local
0,UA 884,Washington,United,Boeing 777-200,2025-09-01 06:15Z,2025-09-01 08:15+02:00
0,U2 1411,Geneva,easyJet,Airbus A320,2025-09-01 06:10Z,2025-09-01 08:10+02:00
0,AZ 1736,Catania,ITA,Airbus A320,2025-09-01 06:30Z,2025-09-01 08:30+02:00
0,W6 1441,Warsaw,Wizz Air,Airbus A320,2025-09-01 06:05Z,2025-09-01 08:05+02:00
0,VY 6100,Barcelona,Vueling,Airbus A321,2025-09-01 06:20Z,2025-09-01 08:20+02:00
